In [8]:
import openpyxl
from openpyxl.styles import Font
from selenium import webdriver
from time import sleep
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from datetime import datetime

browser = webdriver.Chrome(ChromeDriverManager().install())
url = 'https://www.dlsite.com/maniax/ranking/day'
browser.get(url)
elem_login_btn_top = browser.find_element_by_class_name('btn_yes.btn-approval')
elem_login_btn_top.click()

html_source = browser.page_source
soup = BeautifulSoup(html_source, 'html.parser')

In [31]:
data=[]

#商品ページを1つに絞る
Products = soup.find_all('tr', class_=['', 'bg_trans'])


for Product in Products:

    #タイトルの取得
    title=Product.find_all('a')[1]
    title=title.text.strip()

    #サークル名の取得
    circul_name=Product.find_all('a')[2]
    circul_name2=circul_name.text.strip()

    #製品ジャンルの取得
    category_name=Product.find('div',attrs = {'class':'work_category'})
    category_name=category_name.text.strip()

    #販売数の取得
    dl_count_div = Product.find('div', class_='dl_count')
    # エラーチェック（divタグが見つからない場合）
    if dl_count_div is None:
        print("Div tag not found!")
    else:
        # spanタグのテキストを除去して整数型に変換
        sales_count_str = dl_count_div.text.replace('販売数', '').strip()
        sales_count = int(sales_count_str)

    #現状価格の取得
    dl_count_div = Product.find('div', class_='dl_count')
    sales_count_str = dl_count_div.get_text().replace('販売数', '').strip() # テキスト内容を取得し、不要な文字を除去
    sales_count = int(sales_count_str) # 文字列を整数型に変換

    #登録売価の取得
    work_price_span = soup.find('span', class_='work_price discount')
    # エラーチェック（spanタグが見つからない場合）
    if work_price_span is None:
        print("Span tag not found!")
    else:
        # テキスト内容を取得し、カンマと「円」を削除
        work_price_str = work_price_span.text.replace(',', '').replace('円', '')
        work_price_int = int(work_price_str)

    #割引率の取得
    # クラス名が"icon_campaign type_sale"であるspanタグを探す
    discount_percentage_span = soup.find('span', class_='icon_campaign type_sale')
    # エラーチェック（spanタグが見つからない場合）
    if discount_percentage_span is None:
        print("Span tag not found!")
    else:
        # テキスト内容を取得し、%の前の数字部分だけを取り出す
        discount_percentage_str = discount_percentage_span.text.split('%')[0]
        discount_percentage = int(discount_percentage_str)

    #タグの取得
    search_tag = Product.find('dd', class_='search_tag') # クラス名が"search_tag"であるddタグを探す
    tags = [a_tag.text for a_tag in search_tag.find_all('a')] # aタグのテキスト内容を取得し、一つの文字列に結合
    output_string = '／'.join(tags)

    #紹介文の取得
    work_text = Product.find('dd', attrs={'class': 'work_text'}) 
    work_text=work_text.text

    #「音声」「音楽」「体験版」などの有無
    work_genre_tag = Product.find('dd', class_='work_genre') # クラス名が"work_genre"であるddタグを探す
    attributes = [span_tag.text for span_tag in work_genre_tag.find_all('span') if span_tag.text] # spanタグのテキスト内容を取得し、一つの文字列に結合
    output_string = '／'.join(attributes)

    #URL
    product_url_tag = Product.find('a', class_='work_thumb_box')
    product_url = product_url_tag['href'] # href属性を取得

    details = {}
    datum=details
    datum['商品名']=title
    datum['サークル名']=circul_name2
    datum['ジャンル']=category_name
    datum['販売数']=sales_count 
    datum['現状価格']=work_price_int  
    datum['登録売価']=strike_price_int 
    datum['割引率']=discount_percentage
    datum['タグ']=output_string 
    datum['紹介文']=work_text
    datum['音声などの有無']=output_string 
    datum['URL']=product_url
    data.append(datum)
data

AttributeError: ResultSet object has no attribute 'find_all'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [25]:

import pandas as pd
df = pd.DataFrame(data)

import numpy as np 
df.index = np.arange(1, len(df)+1)

import openpyxl

# Excelファイルに書き込むためのファイル名を生成
now = datetime.now()
timestamp_str = now.strftime("%Y%m%d%H%M%S")
filename = f'dlsite_24hour_sales_ranking_data_{timestamp_str}.xlsx'

# Pandasを使用してExcelファイルにデータを書き込み
with pd.ExcelWriter(filename, engine='openpyxl') as writer:
    df.to_excel(writer, sheet_name='Sheet1')


print(f"Excelファイルに書き込みました: {filename}")

Excelファイルに書き込みました: dlsite_24hour_sales_ranking_data_20230818175103.xlsx
